In [1]:
%reset -f
%matplotlib qt

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [89]:
# Carregando a imagem de teste
img = plt.imread('imgs/img5.jpeg')

## convertendo RGB para HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

##mascara verde
mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,160))

##Separando a cor verde da imagem
imask = mask>0
green = np.zeros_like(img, np.uint8)
green[imask] = img[imask]

#deixando a parte verde branca
h,w,_ = green.shape
for e in range(h):
    for i in range(w):
        if green[e][i][0]>0 and green[e][i][1]>0 and green[e][i][1]>0:
            green[e][i]=[255, 255, 255]
            
plt.imshow(green)

In [90]:
#erosão e dilatação para remover ruidos
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(green,kernel,iterations = 4)
dilation = cv2.dilate(erosion,kernel,iterations = 4)
plt.imshow(dilation)
plt.show()

In [4]:
#adicionando blur na imagem
# kernel2 = np.ones((5,5),np.float32)/25
# blur = cv2.bilateralFilter(dilation, 9, 75, 75)
# filter2d

# plt.imshow(np.uint32(blur))
# plt.show()

In [5]:
#adicionando blur na imagem usando gaussian blur
img_blur = cv2.GaussianBlur(dilation,(11,11),cv2.BORDER_DEFAULT)
plt.imshow(img_blur)
plt.show()

In [6]:
# Canny para detecção de bordas
edges = cv2.Canny(img_blur, threshold1=50, threshold2=200,apertureSize=3)
#threshold1=60, threshold2=120
plt.imshow(edges,cmap = 'gray')

In [ ]:
# # compute the median of the single channel pixel intensities
# sigma=0.33
# v = np.median(img_blur)
 
# # apply automatic Canny edge detection using the computed median
# lower = int(max(0, (1.0 - sigma) * v))
# upper = int(min(255, (1.0 + sigma) * v))
# edged = cv2.Canny(img_blur, lower, upper)
# plt.imshow(edged,cmap = 'gray')
# print(lower, upper)

threshold=10,minLineLength=10, maxLineGap=20 bom porém detecta os jogadores tambem <br>
threshold=60,minLineLength=10, maxLineGap=20 tira os players mas tem o campo razoavel

In [ ]:
# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
cdstP_green = np.copy(cdst)

img_edge = np.zeros_like(img, np.uint8)

linesP = cv2.HoughLinesP(edges, rho=0.8, theta=(np.pi / 180), threshold=60,minLineLength=10, maxLineGap=20)
  
linesP
if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(img_edge, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

plt.imshow(img_edge)        

In [ ]:
def distance_to_line(point, line):
    """Get distance between point and line
    https://stackoverflow.com/questions/40970478/python-3-5-2-distance-from-a-point-to-a-line
    """
    px, py = point
    x1, y1, x2, y2 = line
    x_diff = x2 - x1
    y_diff = y2 - y1
    num = abs(y_diff * px - x_diff * py + x2 * y1 - y2 * x1)
    den = math.sqrt(y_diff**2 + x_diff**2)
    return num / den

def get_distance(a_line, b_line):
    """Get all possible distances between each dot of two lines and second line
    return the shortest
    """
    dist1 = distance_to_line(a_line[:2], b_line)
    dist2 = distance_to_line(a_line[2:], b_line)
    dist3 = distance_to_line(b_line[:2], a_line)
    dist4 = distance_to_line(b_line[2:], a_line)

    return min(dist1, dist2, dist3, dist4)

In [ ]:
new_linesP = linesP
to_delete = []
for i in range(len(linesP)):
    l = linesP[i][0]
    for j in range(len(linesP)):
        if j != i:  
            d = get_distance(l, linesP[j][0])
            if(d < 100):
                to_delete.append(j)

In [ ]:
len(to_delete)

In [ ]:
#Novos valores!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
cdstP_green = np.copy(cdst)

img_edge = np.zeros_like(img, np.uint8)

# linesP = cv2.HoughLinesP(edges, 0.8, 1*np.pi / 180, 60, None, 100, 80)
linesP = cv2.HoughLinesP(edges, rho=0.8, theta=(np.pi / 180), threshold=60, minLineLength=100, maxLineGap=80)
    
if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(img_edge, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

plt.imshow(img_edge)       

# https://stackoverflow.com/questions/45531074/how-to-merge-lines-after-houghlinesp implementation

In [ ]:
class HoughBundler:
    '''Clasterize and merge each cluster of cv2.HoughLinesP() output
    a = HoughBundler()
    foo = a.process_lines(houghP_lines, binary_image)
    '''

    def get_orientation(self, line):
        '''get orientation of a line, using its length
        https://en.wikipedia.org/wiki/Atan2
        '''
        orientation = math.atan2(abs((line[0] - line[2])), abs((line[1] - line[3])))
        return math.degrees(orientation)

    def checker(self, line_new, groups, min_distance_to_merge, min_angle_to_merge):
        '''Check if line have enough distance and angle to be count as similar
        '''
        for group in groups:
            # walk through existing line groups
            for line_old in group:
                # check distance
                if self.get_distance(line_old, line_new) < min_distance_to_merge:
                    # check the angle between lines
                    orientation_new = self.get_orientation(line_new)
                    orientation_old = self.get_orientation(line_old)
                    # if all is ok -- line is similar to others in group
                    if abs(orientation_new - orientation_old) < min_angle_to_merge:
                        group.append(line_new)
                        return False
        # if it is totally different line
        return True

    def DistancePointLine(self, point, line):
        """Get distance between point and line
        http://local.wasp.uwa.edu.au/~pbourke/geometry/pointline/source.vba
        """
        px, py = point
        x1, y1, x2, y2 = line

        def lineMagnitude(x1, y1, x2, y2):
            'Get line (aka vector) length'
            lineMagnitude = math.sqrt(math.pow((x2 - x1), 2) + math.pow((y2 - y1), 2))
            return lineMagnitude

        LineMag = lineMagnitude(x1, y1, x2, y2)
        if LineMag < 0.00000001:
            DistancePointLine = 9999
            return DistancePointLine

        u1 = (((px - x1) * (x2 - x1)) + ((py - y1) * (y2 - y1)))
        u = u1 / (LineMag * LineMag)

        if (u < 0.00001) or (u > 1):
            #// closest point does not fall within the line segment, take the shorter distance
            #// to an endpoint
            ix = lineMagnitude(px, py, x1, y1)
            iy = lineMagnitude(px, py, x2, y2)
            if ix > iy:
                DistancePointLine = iy
            else:
                DistancePointLine = ix
        else:
            # Intersecting point is on the line, use the formula
            ix = x1 + u * (x2 - x1)
            iy = y1 + u * (y2 - y1)
            DistancePointLine = lineMagnitude(px, py, ix, iy)

        return DistancePointLine

    def get_distance(self, a_line, b_line):
        """Get all possible distances between each dot of two lines and second line
        return the shortest
        """
        dist1 = self.DistancePointLine(a_line[:2], b_line)
        dist2 = self.DistancePointLine(a_line[2:], b_line)
        dist3 = self.DistancePointLine(b_line[:2], a_line)
        dist4 = self.DistancePointLine(b_line[2:], a_line)

        return min(dist1, dist2, dist3, dist4)

    def merge_lines_pipeline_2(self, lines):
        'Clusterize (group) lines'
        groups = []  # all lines groups are here
        # Parameters to play with
        min_distance_to_merge = 30
        min_angle_to_merge = 30
        # first line will create new group every time
        groups.append([lines[0]])
        # if line is different from existing gropus, create a new group
        for line_new in lines[1:]:
            if self.checker(line_new, groups, min_distance_to_merge, min_angle_to_merge):
                groups.append([line_new])

        return groups

    def merge_lines_segments1(self, lines):
        """Sort lines cluster and return first and last coordinates
        """
        orientation = self.get_orientation(lines[0])

        # special case
        if(len(lines) == 1):
            return [lines[0][:2], lines[0][2:]]

        # [[1,2,3,4],[]] to [[1,2],[3,4],[],[]]
        points = []
        for line in lines:
            points.append(line[:2])
            points.append(line[2:])
        # if vertical
        if 45 < orientation < 135:
            #sort by y
            points = sorted(points, key=lambda point: point[1])
        else:
            #sort by x
            points = sorted(points, key=lambda point: point[0])

        # return first and last point in sorted group
        # [[x,y],[x,y]]
        return [points[0], points[-1]]

    def process_lines(self, lines, img):
        '''Main function for lines from cv.HoughLinesP() output merging
        for OpenCV 3
        lines -- cv.HoughLinesP() output
        img -- binary image
        '''
        lines_x = []
        lines_y = []
        # for every line of cv2.HoughLinesP()
        for line_i in [l[0] for l in lines]:
                orientation = self.get_orientation(line_i)
                # if vertical
                if 45 < orientation < 135:
                    lines_y.append(line_i)
                else:
                    lines_x.append(line_i)

        lines_y = sorted(lines_y, key=lambda line: line[1])
        lines_x = sorted(lines_x, key=lambda line: line[0])
        merged_lines_all = []

        # for each cluster in vertical and horizantal lines leave only one line
        for i in [lines_x, lines_y]:
                if len(i) > 0:
                    groups = self.merge_lines_pipeline_2(i)
                    merged_lines = []
                    for group in groups:
                        merged_lines.append(self.merge_lines_segments1(group))

                    merged_lines_all.extend(merged_lines)

        return merged_lines_all

In [ ]:
test = HoughBundler()

In [ ]:
new_lines = test.process_lines(lines=linesP, img=img_edge)

In [ ]:
for i in range(0, len(new_lines)):
    l = new_lines[i]
    cv2.line(img_edge, (l[0][0], l[0][1]), (l[1][0], l[1][1]), (0,0,255), 3, cv2.LINE_AA)

plt.imshow(img_edge)    

# Homografia

In [56]:
campo_vertical = [0, 18.30, 29.30, 47.62, 58.62, 76.92]

In [57]:
campo_verticalp = [(1080/76.92)*float(x) for x in campo_vertical]

In [58]:
campo_verticalp

[0.0,
 256.9422776911076,
 411.38845553822154,
 668.6115444617784,
 823.0577223088923,
 1080.0]

In [59]:
campo_horizontal = [45, 73.5, 84.5, 90]

In [60]:
campo_horizontalp = [(1920/90)*float(x) for x in campo_horizontal]

In [61]:
campo_horizontalp

[960.0, 1568.0, 1802.6666666666665, 1920.0]

In [62]:
# Item 1
orig = np.array([
        (423, 230),
        (1620, 242),
        (1235, 285),
        (1688, 295),
        (1598, 349),
        (1772, 353),
        (1745, 505),
        (1243, 650)
    # Coloque os pontos aqui
])

In [63]:
dest = np.array(
        [(960, 0),
        (1920, 0),
        (1568, 256),
        (1920, 256),
        (1802, 411),
        (1920, 411),
        (1802, 668),
        #1920, 668,
        (1568, 823),
        #1920, 823,
         #960, 1080, 
        #1920, 1080
])

In [64]:
len(orig) == len(dest)

True

In [86]:
orig_array = np.float32(orig[:, np.newaxis, :])
dest_array = np.float32(dest[:, np.newaxis, :])
Hocv, nsei = cv2.findHomography(dest_array, orig_array) 
# Hocv = Hocv.reshape(-1,1)
Hocv

array([[ 1.70589848e+00, -1.09253079e+00, -1.13663791e+03],
       [ 5.69111951e-02,  5.03994963e-02,  2.05138978e+02],
       [ 1.70232677e-04, -9.16000369e-04,  1.00000000e+00]])

In [175]:
# Item 5
# Crie a imagem final aqui
res = np.zeros((1080,1920, 3), dtype=np.uint8)
Hocv = Hocv.reshape(3,3)

for x in range(res.shape[1]):
    for y in range(res.shape[0]):
        porig = np.array([x,y,1])
        pdest = np.dot(Hocv, porig)
        pfinal = [(pdest[0]/pdest[2]), (pdest[1]/pdest[2])]
        
        xdest = int(pfinal[0])
        ydest = int(pfinal[1])
#         print(x, y)
#         print(ydest, xdest)
        if((0<=xdest<1920) and (0<=ydest<1080)):
#         if(xdest> 0 and ydest>0):
#             print(res[y][x])
            res[y][x] = img[ydest][xdest]
        else:
            res[y][x] = [255,255,255]
        

In [191]:
res_c = res.copy()

In [192]:
imped = [(1578, 0), (1578, 1080)]

In [193]:
res_c=cv2.line(res_c, (imped[0][0], imped[0][1]), (imped[1][0], imped[1][1]), (255,0,0), 3, cv2.LINE_AA)

In [194]:
bola = [(1502, 940), (1920, 539)]

In [195]:
res_c=cv2.line(res_c, (bola[0][0], bola[0][1]), (bola[1][0], bola[1][1]), (0,0,255), 3, cv2.LINE_AA)

In [196]:
dist_bola_campo = math.sqrt( ((bola[0][0]-bola[1][0])**2)+((bola[0][1]-bola[1][1])**2) )
dist_bola_campo = "{0:.3g}".format((90/1920)*dist_bola_campo)

In [197]:
font = cv2.FONT_HERSHEY_SIMPLEX
res_c=cv2.putText(res_c, str(dist_bola_campo + 'm'), (10,450), font, 3, (0, 0, 255), 5, cv2.LINE_AA)

In [198]:
raio_bola = 195.2
res_c=cv2.circle(res_c,bola[0], int(raio_bola), (0,0,255), 0)
plt.imshow(res_c)